In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
import gc
import catboost

In [2]:
training_set = pd.read_csv('training-set.csv')
testing_set = pd.read_csv('testing-set.csv')
claim = pd.read_csv('claim_0702.csv')
policy = pd.read_csv('policy_0702.csv')
duplicate_policy = pd.read_csv('duplicate_policy.csv',header=None,names=['Policy_Number'])
wrong_dbirth = pd.read_csv('wrong_dbirth.csv',header=None,names=['Policy_Number'])

In [3]:
#更正有問題的資料
training_set.loc[training_set['Policy_Number'].isin(duplicate_policy['Policy_Number']),'Next_Premium']=training_set['Next_Premium']/2
policy.loc[policy['Policy_Number'].isin(wrong_dbirth['Policy_Number']),'dbirth']=np.NaN

In [4]:
#從樣本中剔除Insured_Amount1,2,3皆為0,若三者皆為0則直接預測Next_Premium=0
policy=policy[(policy['Insured_Amount1']!=0) | (policy['Insured_Amount2']!=0) | (policy['Insured_Amount3']!=0)]

In [5]:
baseline=policy[['Policy_Number','Premium']]

Label_1=[s+'_'+l for s in baseline.columns.tolist() if s!='Policy_Number' for l in ['SUM','MEAN','COUNT']]
baseline=baseline.groupby('Policy_Number').agg(['sum','mean','count']).reset_index()
baseline.columns=['Policy_Number']+Label_1

In [6]:
#產生變數: Premium_Refund
policy['Premium_Refund']=policy['Premium']
policy.loc[policy['Coverage_Deductible_if_applied']>0,'Premium_Refund']=0
policy.loc[policy['Coverage_Deductible_if_applied']<0,'Premium']=0

In [7]:
training_set_baseline=training_set[['Policy_Number','Next_Premium']].\
merge(baseline,on='Policy_Number',how='left')

In [8]:
training_set_baseline['segment']=0

training_set_baseline.loc[training_set_baseline['Next_Premium']!=0,'segment']=1

In [9]:
training_set_baseline2=training_set_baseline.drop(['Next_Premium','Next_Premium','Premium_SUM','Premium_MEAN','Premium_COUNT'],axis=1)
training_set_baseline3=training_set_baseline.drop(['Next_Premium','Next_Premium','Premium_MEAN','Premium_COUNT'],axis=1)

Insured's_IＤ衍生變數

In [10]:
policyID=policy.iloc[:,0:2].drop_duplicates()
policyID.columns=['Policy_Number','Insureds_ID']

In [11]:
policy_amt=policy[['Policy_Number','Insured_Amount1','Insured_Amount2','Insured_Amount3','Premium']]
policy_amt=policy_amt.groupby('Policy_Number').agg(['sum']).reset_index()
policy_amt.columns=['Policy_Number','Insured_Amount1_ID','Insured_Amount2_ID','Insured_Amount3_ID','Premium_ID']

In [12]:
policy_amt.shape

(337188, 5)

In [13]:
policyID_count=policyID.iloc[:,1].value_counts().to_frame().reset_index()
policyID_count=policyID_count[policyID_count['Insureds_ID']>1]

In [14]:
policyID_count=policyID.iloc[:,1].value_counts().to_frame().reset_index()
policyID_count=policyID_count[policyID_count['Insureds_ID']>1]
policyID_count=policyID_count.rename(columns={'index':'Insureds_ID','Insureds_ID':'Insureds_ID_count'})

In [15]:
policyID=policyID.merge(policyID_count,on='Insureds_ID',how='inner')\
.merge(policy_amt,on='Policy_Number',how='left')

In [16]:
policyID2=policyID.iloc[:,1:]

In [17]:
policyID2=policyID.iloc[:,1:]

Label_1=[s+'_'+l for s in policyID2.columns.tolist() if s!='Insureds_ID' for l in ['SUM','MEAN','MAX']]
policyID2=policyID2.groupby('Insureds_ID').agg(['sum','mean','max']).reset_index()
policyID2.columns=['Insureds_ID']+Label_1

In [18]:
policyID2=policyID.merge(policyID2,on='Insureds_ID',how='left')\
.drop(['Insureds_ID_count_SUM','Insureds_ID_count_MEAN','Insureds_ID_count_MAX'],axis=1)

In [19]:
policyID2['Insured_Amount1_ID_ratio']=1/policyID2['Insureds_ID_count']
policyID2['Insured_Amount2_ID_ratio']=1/policyID2['Insureds_ID_count']
policyID2['Insured_Amount3_ID_ratio']=1/policyID2['Insureds_ID_count']
policyID2['Premium_ID_ratio']=1/policyID2['Insureds_ID_count']

In [20]:
policyID2.loc[policyID2['Insured_Amount1_ID_SUM']!=0,'Insured_Amount1_ID_ratio']=\
policyID2['Insured_Amount1_ID']/policyID2['Insured_Amount1_ID_SUM']

policyID2.loc[policyID2['Insured_Amount2_ID_SUM']!=0,'Insured_Amount2_ID_ratio']=\
policyID2['Insured_Amount2_ID']/policyID2['Insured_Amount2_ID_SUM']

policyID2.loc[policyID2['Insured_Amount3_ID_SUM']!=0,'Insured_Amount3_ID_ratio']=\
policyID2['Insured_Amount3_ID']/policyID2['Insured_Amount3_ID_SUM']

policyID2.loc[policyID2['Premium_ID_SUM']!=0,'Premium_ID_ratio']=\
policyID2['Premium_ID']/policyID2['Premium_ID_SUM']

挑選可能有用變數

In [21]:
policy_cif=policy[[
#car var
'Policy_Number','Vehicle_identifier',
'Vehicle_Make_and_Model1','Vehicle_Make_and_Model2',\
'Manafactured_Year_and_Month',"Engine_Displacement_(Cubic_Centimeter)",\
'Imported_or_Domestic_Car','Coding_of_Vehicle_Branding_&_Type',\
'qpt','fpt','fequipment1','fequipment2','fequipment3','fequipment4',\
'fequipment5','fequipment6','fequipment9','nequipment9',\
'Replacement_cost_of_insured_vehicle','Distribution_Channel',
'lia_class','plia_acc','pdmg_acc',\
#personal var
'Multiple_Products_with_TmNewa_(Yes_or_No?)',\
'fsex','fmarriage','fassured','ibirth','dbirth',\
'Prior_Policy_Number','Cancellation',\
'aassured_zip','iply_area']]
policy_cif=policy_cif.drop_duplicates()
print(policy_cif.shape)
#"Insured's_ID" is not included

(337188, 33)


移除前筆保單,車牌
equipment相關變數（值太少）

In [22]:
policy_cif=policy_cif.drop(['Prior_Policy_Number','Vehicle_identifier'],axis=1)
policy_cif=policy_cif.drop(["fpt"],axis=1) #fpt has only one value
policy_cif=policy_cif.drop([
'fequipment1','fequipment2','fequipment3',\
'fequipment4','fequipment5','fequipment6',\
'fequipment9','nequipment9'],axis=1) #only few values

整理類別型變數（不含保單相關變數）

In [23]:
#定義要處理的欄位
object_list=[
'Vehicle_Make_and_Model1',\
'Vehicle_Make_and_Model2',\
"Imported_or_Domestic_Car",\
"Coding_of_Vehicle_Branding_&_Type",\
"Distribution_Channel",\
'fassured',\
'fsex',\
'fmarriage',\
'aassured_zip',\
'iply_area',\
]
#remove iply_area, aassured_zip,Distribution_Channel

In [24]:
#處理空值＆指定欄位型態
for var in object_list:
    policy_cif[var]=policy_cif[var].replace(' ',np.nan)
    policy_cif[var]=policy_cif[var].astype('object')

In [25]:
#廠牌名稱,主通路代號做特別處理,把類別出現一千次以下的併一群
transform_list=['Vehicle_Make_and_Model1','Distribution_Channel',
                'Vehicle_Make_and_Model2',
                'Coding_of_Vehicle_Branding_&_Type',
               'aassured_zip']#,'iply_area']
for var in transform_list:
    comb=policy_cif[['Policy_Number',var]].groupby(var).count().\
    sort_values(by=['Policy_Number'],ascending=False).reset_index().rename(columns={'Policy_Number':var+'_Count'})
    comb[var+'_New']=comb[var]
    comb.loc[comb[var+'_Count']<1000,var+'_New']='others'
    comb=comb.drop([var+'_Count'],axis=1)
    policy_cif=policy_cif.merge(comb,on=var,how='left')
    policy_cif=policy_cif.drop([var],axis=1)
    policy_cif[var+'_New']=policy_cif[var+'_New'].astype('object')

In [26]:
policy_cif=policy_cif.rename(columns={'iply_area':'iply_area_New',
                             'Vehicle_Make_and_Model1':'Vehicle_Make_and_Model1_New',
                             'Distribution_Channel':'Distribution_Channel_New',
                             'Vehicle_Make_and_Model2':'Vehicle_Make_and_Model2_New',
                             'Coding_of_Vehicle_Branding_&_Type':'Coding_of_Vehicle_Branding_&_Type_New',
                             'aassured_zip':'aassured_zip_New'})

In [27]:
#處理：非繼承自前保單
policy_cif['Cancellation']=policy_cif['Cancellation'].replace(' ',0)
policy_cif['Cancellation']=policy_cif['Cancellation'].replace('Y',1)
policy_cif['Cancellation']=policy_cif['Cancellation'].astype('float64')
#處理：製造年份
policy_cif['Manafactured_Year_and_Month']=2017-policy_cif['Manafactured_Year_and_Month']
#處理：出生日期,要保人出生日期
policy_cif['ibirth']=policy_cif['ibirth'].str.extract('(\d{4})',expand=False)
policy_cif['dbirth']=policy_cif['dbirth'].str.extract('(\d{4})',expand=False)
policy_cif['ibirth']=2017-policy_cif['ibirth'].astype('float64')
policy_cif['dbirth']=2017-policy_cif['dbirth'].astype('float64')

In [28]:
#類別型變數處理 供lightgbm使用
def object_to_int(dataset,column):
    object_list=dataset[column].unique().tolist()
    for i,value in enumerate(object_list):
        dataset.loc[dataset[column]==value,column]=i+1
    dataset[column]=dataset[column].fillna(0)
    dataset[column]=dataset[column].astype('int')
    return dataset[column]

In [29]:
policy_cif['Imported_or_Domestic_Car']=policy_cif['Imported_or_Domestic_Car'].astype('int')

policy_cif['fsex']=policy_cif['fsex'].fillna(0)
policy_cif['fsex']=policy_cif['fsex'].astype('int')

policy_cif['fmarriage']=policy_cif['fmarriage'].fillna(0)
policy_cif['fmarriage']=policy_cif['fmarriage'].astype('int')

policy_cif['fassured']=policy_cif['fassured'].fillna(0)
policy_cif['fassured']=policy_cif['fassured'].astype('int')

policy_cif['iply_area_New']=object_to_int(policy_cif,'iply_area_New')
policy_cif['Vehicle_Make_and_Model1_New']=object_to_int(policy_cif,'Vehicle_Make_and_Model1_New')
policy_cif['Distribution_Channel_New']=object_to_int(policy_cif,'Distribution_Channel_New')

policy_cif['Vehicle_Make_and_Model2_New']=object_to_int(policy_cif,'Vehicle_Make_and_Model2_New')
policy_cif['Coding_of_Vehicle_Branding_&_Type_New']=object_to_int(policy_cif,'Coding_of_Vehicle_Branding_&_Type_New')
policy_cif['aassured_zip_New']=object_to_int(policy_cif,'aassured_zip_New')

整理數值型變數（保單相關）

In [30]:
#處理自負額
Insurance_Coverage_no_cov = ['09I', '10A', '14E', '15F', 
                             '15O', '20B', '20K', '29K', 
                             '32N', '33F', '33O', '56K', 
                             '65K']
flt = policy['Insurance_Coverage'].isin(Insurance_Coverage_no_cov)
idx = policy[flt].index

# It's ok
policy.loc[idx, 'Coverage_Deductible_if_applied'] = 0

In [31]:
# Warning is ok
sum_insured = (policy.Insured_Amount1 + policy.Insured_Amount2 + policy.Insured_Amount3)

flt = (policy['Coverage_Deductible_if_applied'] == 10)
policy.loc[flt, 'Coverage_Deductible_if_applied'] = sum_insured*0.1

flt = (policy['Coverage_Deductible_if_applied'] == 20)
policy.loc[flt, 'Coverage_Deductible_if_applied'] = sum_insured*0.2


flt = (policy['Coverage_Deductible_if_applied'] == 1)
policy.loc[flt, 'Coverage_Deductible_if_applied'] = 5000

flt = (policy['Coverage_Deductible_if_applied'] == 2)
policy.loc[flt, 'Coverage_Deductible_if_applied'] = 6500

flt = (policy['Coverage_Deductible_if_applied'] == 3)
policy.loc[flt, 'Coverage_Deductible_if_applied'] = 7666

# 退保的部分也要做轉換
flt = (policy['Coverage_Deductible_if_applied'] == -10)
policy.loc[flt, 'Coverage_Deductible_if_applied'] = sum_insured*0.1

flt = (policy['Coverage_Deductible_if_applied'] == -20)
policy.loc[flt, 'Coverage_Deductible_if_applied'] = sum_insured*0.2


flt = (policy['Coverage_Deductible_if_applied'] == -1)
policy.loc[flt, 'Coverage_Deductible_if_applied'] = -5000

flt = (policy['Coverage_Deductible_if_applied'] == -2)
policy.loc[flt, 'Coverage_Deductible_if_applied'] = -6500

flt = (policy['Coverage_Deductible_if_applied'] == -3)
policy.loc[flt, 'Coverage_Deductible_if_applied'] = -7666



In [32]:
policy_amount=policy[['Policy_Number','Insured_Amount1','Insured_Amount2','Insured_Amount3','Coverage_Deductible_if_applied','Premium']]

Label_1=[s+'_'+l for s in policy_amount.columns.tolist() if s!='Policy_Number' for l in ['MEAN','COUNT','SUM','MEDIAN','MAX']]
avg_policy_amount=policy_amount.groupby('Policy_Number').agg(['mean','count','sum','median','max']).reset_index()
avg_policy_amount.columns=['Policy_Number']+Label_1
avg_policy_amount=avg_policy_amount.drop(['Insured_Amount1_COUNT','Insured_Amount2_COUNT','Insured_Amount3_COUNT','Coverage_Deductible_if_applied_COUNT'],axis=1)
avg_policy_amount.head()

,Policy_Number,Insured_Amount1_MEAN,Insured_Amount1_SUM,Insured_Amount1_MEDIAN,Insured_Amount1_MAX,Insured_Amount2_MEAN,Insured_Amount2_SUM,Insured_Amount2_MEDIAN,Insured_Amount2_MAX,Insured_Amount3_MEAN,...,Insured_Amount3_MAX,Coverage_Deductible_if_applied_MEAN,Coverage_Deductible_if_applied_SUM,Coverage_Deductible_if_applied_MEDIAN,Coverage_Deductible_if_applied_MAX,Premium_MEAN,Premium_COUNT,Premium_SUM,Premium_MEDIAN,Premium_MAX
0,000061141b237e8619efedcd6939fddeff05b9a5,166666.666667,500000,0,500000,0.000000,0,0,0,700000,...,1000000,0.000000,0.0,0.0,0.0,528.666667,3,1586,660.0,727
1,0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5,314285.714286,2200000,0,2000000,285714.285714,2000000,0,2000000,3935142,...,24000000,3885.714286,27200.0,0.0,27200.0,1304.571429,7,9132,661.0,5352
2,0001221abee6f59d6d40eeb1ba9ffa0f4f92b36b,1000000.000000,3000000,0,3000000,0.000000,0,0,0,3100000,...,6000000,0.000000,0.0,0.0,0.0,953.666667,3,2861,801.0,1280
3,000148bbf40f28bfc860c0d272aadc429e19e2d1,500000.000000,1000000,500000,1000000,0.000000,0,0,0,6100000,...,12000000,0.000000,0.0,0.0,0.0,864.000000,2,1728,864.0,891
4,0001ad0ad0618f6d2e2c1875105adce6bcd77c2c,336666.666667,1010000,10000,1000000,0.000000,0,0,0,783333,...,2000000,0.000000,0.0,0.0,0.0,731.666667,3,2195,885.0,1024


整理類別型變數（保單相關）

In [33]:
policy_category=policy[['Policy_Number','Main_Insurance_Coverage_Group','Insurance_Coverage']]
policy_cat_index=policy_category['Policy_Number']
policy_cat=pd.get_dummies(policy_category.drop(['Policy_Number'],axis=1))
policy_category=pd.concat([policy_cat_index,policy_cat],axis=1)

In [34]:
Label_1=[s+'_'+l for s in policy_category.columns.tolist() if s!='Policy_Number' for l in ['SUM']]
avg_policy_category=policy_category.groupby('Policy_Number').agg(['sum']).reset_index()
avg_policy_category.columns=['Policy_Number']+Label_1
avg_policy_category.head()

,Policy_Number,Main_Insurance_Coverage_Group_竊盜_SUM,Main_Insurance_Coverage_Group_車損_SUM,Main_Insurance_Coverage_Group_車責_SUM,Insurance_Coverage_00I_SUM,Insurance_Coverage_01A_SUM,Insurance_Coverage_01J_SUM,Insurance_Coverage_02K_SUM,Insurance_Coverage_03L_SUM,Insurance_Coverage_04M_SUM,...,Insurance_Coverage_65K_SUM,Insurance_Coverage_66C_SUM,Insurance_Coverage_66L_SUM,Insurance_Coverage_67D_SUM,Insurance_Coverage_68E_SUM,Insurance_Coverage_68N_SUM,Insurance_Coverage_70G_SUM,Insurance_Coverage_70P_SUM,Insurance_Coverage_71H_SUM,Insurance_Coverage_72@_SUM
0,000061141b237e8619efedcd6939fddeff05b9a5,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0000d65ef9b4e9e5e9603617bd5fb8497b7f02b5,2,2,3,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0001221abee6f59d6d40eeb1ba9ffa0f4f92b36b,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,000148bbf40f28bfc860c0d272aadc429e19e2d1,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0001ad0ad0618f6d2e2c1875105adce6bcd77c2c,0,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


整理claim表

In [35]:
object_list2=[\
'Nature_of_the_claim',\
"Driver's_Gender",\
"Driver's_Relationship_with_Insured",\
'Marital_Status_of_Driver',\
'Cause_of_Loss',\
'Coverage',\
"Claim_Status_(close,_open,_reopen_etc)",\
'Accident_area',\
'Accident_Time'
]
value_list=[\
'Paid_Loss_Amount',\
'paid_Expenses_Amount',\
'Salvage_or_Subrogation?',\
'At_Fault?',\
'Deductible',\
'number_of_claimants',\
'DOB_of_Driver',\
'Accident_Date'\
]

In [36]:
#處理空值＆指定欄位型態
for var in object_list2:
    claim[var]=claim[var].replace(' ',np.nan)
    claim[var]=claim[var].astype('object')

In [37]:
#處理：肇事駕駛出生日,出險日期
claim['DOB_of_Driver']=claim['DOB_of_Driver'].str.extract('(\d{4})',expand=False)
claim['Accident_Date']=claim['Accident_Date'].str.extract('(\d{4})',expand=False)
claim['DOB_of_Driver']=2017-claim['DOB_of_Driver'].astype('float64')
claim['Accident_Date']=2017-claim['Accident_Date'].astype('float64')

In [38]:
#處理：出險時間
claim['Accident_Time']=claim['Accident_Time'].str.extract('(\d{2})',expand=False)
claim['Accident_Time']=claim['Accident_Time'].astype('object')

In [39]:
#類別型變數轉dummy
claim_cat=claim[object_list2]
claim_index=claim['Policy_Number']
claim_cat=pd.get_dummies(claim_cat)
claim_cat=pd.concat([claim_index,claim_cat],axis=1)

In [40]:
#整理dummy
Label_1=[s+'_'+l for s in claim_cat.columns.tolist() if s!='Policy_Number' for l in ['SUM']]
avg_claim_cat=claim_cat.groupby('Policy_Number').agg(['sum']).reset_index()
avg_claim_cat.columns=['Policy_Number']+Label_1
avg_claim_cat.head()

,Policy_Number,Nature_of_the_claim_1_SUM,Nature_of_the_claim_2_SUM,Driver's_Gender_1_SUM,Driver's_Gender_2_SUM,Driver's_Relationship_with_Insured_1_SUM,Driver's_Relationship_with_Insured_2_SUM,Driver's_Relationship_with_Insured_3_SUM,Driver's_Relationship_with_Insured_4_SUM,Driver's_Relationship_with_Insured_5_SUM,...,Accident_Time_14_SUM,Accident_Time_15_SUM,Accident_Time_16_SUM,Accident_Time_17_SUM,Accident_Time_18_SUM,Accident_Time_19_SUM,Accident_Time_20_SUM,Accident_Time_21_SUM,Accident_Time_22_SUM,Accident_Time_23_SUM
0,0004a3f30deef9fc8f9f5cb334b8bec8d79c7453,3,4,0,7,7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0009049ed3ac539074a3cbfadb59cac6aec2fd98,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,000949143e8dc7d5f570cb8c3b6aa51c8f8ac9ad,3,0,0,3,0,0,0,0,3,...,2,0,0,0,1,0,0,0,0,0
3,0009dfd25805f44c7fa82c1ddb010eb3a05612f6,3,0,0,3,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
4,000a5d472fa501c2e9d771dfae4af646f281f97d,4,0,4,0,0,0,0,0,4,...,0,0,4,0,0,0,0,0,0,0


In [41]:
#整理數值型變數
claim_value=claim[value_list]
claim_value=pd.concat([claim_index,claim_value],axis=1)

value_list_count=[\
#'Paid_Loss_Amount_COUNT',\# left one var dont' drop
'paid_Expenses_Amount_COUNT',\
'Salvage_or_Subrogation?_COUNT',\
'At_Fault?_COUNT',\
'Deductible_COUNT',\
'number_of_claimants_COUNT',\
'DOB_of_Driver_COUNT',\
'Accident_Date_COUNT']

Label_1=[s+'_'+l for s in claim_value.columns.tolist() if s!='Policy_Number' for l in ['MEAN','COUNT','MEDIAN','MAX']]
avg_claim_value=claim_value.groupby('Policy_Number').agg(['mean','count','median','max']).reset_index()
avg_claim_value.columns=['Policy_Number']+Label_1
avg_claim_value=avg_claim_value.drop(value_list_count,axis=1)
avg_claim_value.head()

,Policy_Number,Paid_Loss_Amount_MEAN,Paid_Loss_Amount_COUNT,Paid_Loss_Amount_MEDIAN,Paid_Loss_Amount_MAX,paid_Expenses_Amount_MEAN,paid_Expenses_Amount_MEDIAN,paid_Expenses_Amount_MAX,Salvage_or_Subrogation?_MEAN,Salvage_or_Subrogation?_MEDIAN,...,Deductible_MAX,number_of_claimants_MEAN,number_of_claimants_MEDIAN,number_of_claimants_MAX,DOB_of_Driver_MEAN,DOB_of_Driver_MEDIAN,DOB_of_Driver_MAX,Accident_Date_MEAN,Accident_Date_MEDIAN,Accident_Date_MAX
0,0004a3f30deef9fc8f9f5cb334b8bec8d79c7453,11371.428571,7,0.0,79600,564.285714,0.0,3000,6114.285714,0.0,...,0,1.787000,1.787,1.787,67.0,67.0,67.0,1.0,1.0,1.0
1,0009049ed3ac539074a3cbfadb59cac6aec2fd98,40315.000000,1,40315.0,40315,0.000000,0.0,0,0.000000,0.0,...,0,1.787000,1.787,1.787,43.0,43.0,43.0,1.0,1.0,1.0
2,000949143e8dc7d5f570cb8c3b6aa51c8f8ac9ad,9613.666667,3,10555.0,18286,0.000000,0.0,0,0.000000,0.0,...,0,1.916333,1.981,1.981,41.0,41.0,41.0,2.0,2.0,2.0
3,0009dfd25805f44c7fa82c1ddb010eb3a05612f6,4498.666667,3,0.0,13496,233.333333,0.0,700,0.000000,0.0,...,0,1.981000,1.981,1.981,53.0,53.0,53.0,1.0,1.0,1.0
4,000a5d472fa501c2e9d771dfae4af646f281f97d,5129.000000,4,2108.0,16300,210.000000,70.0,700,0.000000,0.0,...,0,1.981000,1.981,1.981,33.0,33.0,33.0,1.0,1.0,1.0


In [42]:
#移除太多類別的變數：廠牌名稱,車型代號,郵遞區號
policy_cif=policy_cif.drop([\
'Vehicle_Make_and_Model2_New',\
'Coding_of_Vehicle_Branding_&_Type_New',\
'aassured_zip_New'],axis=1)
#'Distribution_Channel_New',\
#'iply_area_New'

#remove iply_area, aassured_zip,Distribution_Channel suggested by 堂軒, 
#上述做法未奏效,改回原做法

合併母體

In [43]:
training_set_var=training_set.\
merge(policy_cif,on='Policy_Number',how='left').\
merge(avg_policy_amount,on='Policy_Number',how='left').\
merge(avg_policy_category,on='Policy_Number',how='left').\
merge(avg_claim_cat,on='Policy_Number',how='left').\
merge(avg_claim_value,on='Policy_Number',how='left')

testing_set_var=testing_set.\
merge(policy_cif,on='Policy_Number',how='left').\
merge(avg_policy_amount,on='Policy_Number',how='left').\
merge(avg_policy_category,on='Policy_Number',how='left').\
merge(avg_claim_cat,on='Policy_Number',how='left').\
merge(avg_claim_value,on='Policy_Number',how='left')

In [44]:
policyID2=policyID2\
[['Policy_Number',
  'Insureds_ID_count',
 'Insured_Amount1_ID',
 'Insured_Amount2_ID',
 'Insured_Amount3_ID',
 'Premium_ID',
 'Insured_Amount1_ID_SUM',
 'Insured_Amount1_ID_MEAN',
 'Insured_Amount1_ID_MAX',
 'Insured_Amount2_ID_SUM',
 'Insured_Amount2_ID_MEAN',
 'Insured_Amount2_ID_MAX',
 'Insured_Amount3_ID_SUM',
 'Insured_Amount3_ID_MEAN',
 'Insured_Amount3_ID_MAX',
 'Premium_ID_SUM',
 'Premium_ID_MEAN',
 'Premium_ID_MAX',
 'Insured_Amount1_ID_ratio',
 'Insured_Amount2_ID_ratio',
 'Insured_Amount3_ID_ratio',
 'Premium_ID_ratio']]

In [45]:
#rule base排掉Insured amount1,2,3全0
training_set_var_0=training_set_var[training_set_var['lia_class'].isnull()==True]
training_set_var=training_set_var[training_set_var['lia_class'].isnull()!=True]

testing_set_var_0=testing_set_var[testing_set_var['lia_class'].isnull()==True]
testing_set_var=testing_set_var[testing_set_var['lia_class'].isnull()!=True]

In [46]:
training_set_var.shape,training_set_var_0.shape,testing_set_var_0.shape,testing_set_var.shape

((202191, 255), (8572, 255), (5513, 255), (134997, 255))

In [47]:
#classifier的Ｘ跟Ｙ
seg2=training_set_var.merge(training_set_baseline2,on=['Policy_Number'],how='left')

training_set_var2=training_set_var.merge(policyID2,on=['Policy_Number'],how='left')

testing_set_var2=testing_set_var.merge(policyID2,on=['Policy_Number'],how='left')

training_Y=seg2['segment']
training_X=training_set_var2.iloc[:,2:]
testing_X=testing_set_var2.iloc[:,2:]

In [48]:
#Regressor1的Ｘ跟Ｙ:預測log(Next_Premium+1)
training_set_var2=training_set_var.merge(training_set_baseline2,on=['Policy_Number'],how='left')
training_set_var2=training_set_var2[training_set_var2['segment']!=0]

training_YY=training_set_var2['Next_Premium'].apply(lambda x:np.log(x+1))
training_set_var2=training_set_var2.drop(['segment'],axis=1)
training_XX=training_set_var2.iloc[:,2:]

In [49]:
#Regressor2的Ｘ跟Ｙ:預測Premium的變化量
training_set_var2=training_set_var.merge(training_set_baseline2,on=['Policy_Number'],how='left')
training_set_var2=training_set_var2[training_set_var2['segment']!=0]

training_YY_2=training_set_var2['Next_Premium']-training_set_var2['Premium_SUM']
training_set_var2=training_set_var2.drop(['segment'],axis=1)
training_XX_2=training_set_var2.iloc[:,2:]


In [50]:
#Regressor3的Ｘ跟Ｙ:直接預測Premium
training_set_var2=training_set_var.merge(training_set_baseline2,on=['Policy_Number'],how='left')
training_set_var2=training_set_var2[training_set_var2['segment']!=0]

training_YY_3=training_set_var2['Next_Premium']
training_set_var2=training_set_var2.drop(['segment'],axis=1)
training_XX_3=training_set_var2.iloc[:,2:]

In [51]:
print(training_X.shape,testing_X.shape,training_XX.shape,training_XX_2.shape)

(202191, 274) (134997, 274) (164537, 253) (164537, 253)


In [52]:
print(training_X.shape,testing_X.shape,training_XX.shape,training_XX_2.shape)

(202191, 274) (134997, 274) (164537, 253) (164537, 253)


In [53]:
cat_features=['Imported_or_Domestic_Car',
              'fsex','fmarriage','fassured',
              'iply_area_New',
              'Vehicle_Make_and_Model1_New',
              'Distribution_Channel_New'
              #'Vehicle_Make_and_Model2_New',#
              #'Coding_of_Vehicle_Branding_&_Type_New',#
              #'aassured_zip_New'#
             ]

In [54]:
# 自定義log(Next_Premium)的eval
def log_error(preds, train_data):
    train_data=np.exp(train_data.get_label())-1
    preds=np.exp(preds)-1
    return 'error',mean_absolute_error(preds,train_data) , False

In [55]:
import lightgbm
train_data_XX = lightgbm.Dataset(training_XX, label=training_YY, categorical_feature=cat_features)

params={'objective':'regression_l1',
        'metrics':'mae',
        'learning_rate':0.02,
        'num_leaves':32,
        'colsample_bytree':0.7,
        'subsample':0.7,
        'max_depth':7,
        'min_child_weight':1}

clf_XX_gbm = lightgbm.train(params,train_data_XX,num_boost_round=1500,categorical_feature=cat_features,feval=log_error)

In [56]:
mean_absolute_error(training_YY,clf_XX_gbm.predict(training_XX))

0.11326013522902971

In [57]:
import lightgbm
train_data_XX_2 = lightgbm.Dataset(training_XX_2, label=training_YY_2, categorical_feature=cat_features)

params={'objective':'regression_l1',
        'metric': 'mae',
        'learning_rate':0.01,
        'num_leaves':32,
        'colsample_bytree':0.7,
        'subsample':0.7,
        'max_depth':7,
        'min_child_weight':1}

clf_XX_2_gbm = lightgbm.train(params,train_data_XX_2,num_boost_round=840,categorical_feature=cat_features)

In [58]:
#clf_XX_2_gbm.predict(training_XX_2)
mean_absolute_error(training_YY_2,clf_XX_2_gbm.predict(training_XX_2))

849.80189351284275

In [59]:
import lightgbm
train_data_XX_3 = lightgbm.Dataset(training_XX_3, label=training_YY_3, categorical_feature=cat_features)

params={'objective':'regression_l1',
        'metric': 'mae',
        'learning_rate':0.01,
        'num_leaves':32,
        'colsample_bytree':0.7,
        'subsample':0.7,
        'max_depth':7,
        'min_child_weight':1}

clf_XX_3_gbm = lightgbm.train(params,train_data_XX_3,num_boost_round=900,categorical_feature=cat_features)

In [60]:
#clf_XX_3_gbm.predict(training_XX_3)
mean_absolute_error(training_YY_3,clf_XX_3_gbm.predict(training_XX_3))

882.71415482765633

建立三個regressor的ensemble線性迴歸

In [61]:
seg2_ensemble=training_set_var.merge(training_set_baseline2,on=['Policy_Number'],how='left')
#seg2_ensemble.loc[seg2_ensemble['segment']!=0,'segment']=1
seg2_ensemble=seg2_ensemble[seg2_ensemble['segment']==1]

In [65]:
seg2_ensemble2=seg2_ensemble.copy()
#產生diff版的預測值
seg2_ensemble2['Next_Premium_Predict_diff']=clf_XX_2_gbm.predict(seg2_ensemble.\
drop(['Policy_Number','Next_Premium','segment'],axis=1))

#seg2_ensemble2=seg2_ensemble2.merge(baseline,on='Policy_Number',how='left')

seg2_ensemble2['Next_Premium_Predict_diff']=seg2_ensemble2['Premium_SUM']+seg2_ensemble2['Next_Premium_Predict_diff']
seg2_ensemble2['Next_Premium_Predict_diff']=seg2_ensemble2['Next_Premium_Predict_diff'].apply(lambda x: x if x>0 else 0)

In [66]:
#產生log版的預測值
seg2_ensemble2['Next_Premium_Predict_log']=clf_XX_gbm.predict(seg2_ensemble.\
drop(['Policy_Number','Next_Premium','segment'],axis=1))

seg2_ensemble2['Next_Premium_Predict_log']=seg2_ensemble2['Next_Premium_Predict_log'].apply(lambda x: np.e**x-1)

In [67]:
#產生直接預測值
seg2_ensemble2['Next_Premium_Predict_dir']=clf_XX_3_gbm.predict(seg2_ensemble.\
drop(['Policy_Number','Next_Premium','segment'],axis=1))

In [68]:
training_XX_ensemble=seg2_ensemble2[['Next_Premium_Predict_diff','Next_Premium_Predict_log','Next_Premium_Predict_dir']]
training_YY_ensemble=seg2_ensemble2['Next_Premium']

In [69]:
from sklearn.linear_model import LinearRegression

In [70]:
clf_linear=LinearRegression(fit_intercept= False)
clf_linear.fit(training_XX_ensemble,training_YY_ensemble)

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

In [71]:
print(clf_linear.coef_)

[ 0.30239262  1.3177996  -0.6280661 ]


訓練classifier

In [73]:
import lightgbm
train_data_X = lightgbm.Dataset(training_X,label=training_Y,categorical_feature=cat_features)

In [74]:
params={'objective':'multiclass',
        'num_class':2,
        'metric': 'multi_logloss',
        'learning_rate':0.01,
        'num_leaves':32,
        'colsample_bytree':0.7,
        'subsample':0.7,
        'max_depth':7,
        'min_child_weight':1}

clf_gbm = lightgbm.train(params,train_data_X,num_boost_round=1100,categorical_feature=cat_features)

In [76]:
#看lightGBM分類器在train的logloss
training_set_var_new=training_set_var.copy()

segment_predict_prob=clf_gbm.predict(training_X)
segment_logloss=training_set_var_new.merge(training_set_baseline2,on='Policy_Number',how='left')

from sklearn.metrics import log_loss
log_loss(segment_logloss['segment'],segment_predict_prob)

0.4357365726164521

組合所有模型,看train的表現

In [79]:
segment_predict_prob=clf_gbm.predict(training_X)

segment_predict_prob=pd.DataFrame(segment_predict_prob,columns=['Prob0','Prob1'])

segment_predict=segment_predict_prob['Prob0'].apply(lambda x:0 if x>0.5 else 1)

training_set_var_new=training_set_var.copy().reset_index(drop=True)

training_set_var_new['segment_predict']=segment_predict

In [82]:
training_1=training_set_var_new[training_set_var_new['segment_predict']==0].reset_index(drop=True)
training_2=training_set_var_new[training_set_var_new['segment_predict']==1].reset_index(drop=True)

In [85]:
training_1['Next_Premium_Predict']=0

training_3=training_set_var_0.copy()
training_3['Next_Premium_Predict']=0

In [83]:
training_X.shape,training_set_var_new.shape,segment_predict.shape,training_set_var.shape

((202191, 274), (202191, 256), (202191,), (202191, 255))

In [86]:
training_1.shape,training_2.shape,training_3.shape

((3899, 257), (198292, 256), (8572, 256))

In [88]:
training_2_ensemble=training_2.copy()
#產生diff版的預測值
training_2_ensemble['Next_Premium_Predict_diff']=clf_XX_2_gbm.predict(training_2.\
drop(['Policy_Number','Next_Premium','segment_predict'],axis=1))

training_2_ensemble['Next_Premium_Predict_diff']=training_2_ensemble['Premium_SUM']+training_2_ensemble['Next_Premium_Predict_diff']
training_2_ensemble['Next_Premium_Predict_diff']=training_2_ensemble['Next_Premium_Predict_diff'].apply(lambda x: x if x>0 else 0)

In [89]:
#產生log版的預測值
training_2_ensemble['Next_Premium_Predict_log']=clf_XX_gbm.predict(training_2.\
drop(['Policy_Number','Next_Premium','segment_predict'],axis=1))

training_2_ensemble['Next_Premium_Predict_log']=training_2_ensemble['Next_Premium_Predict_log'].apply(lambda x: np.e**x-1)

In [90]:
#產生dir版的預測值
training_2_ensemble['Next_Premium_Predict_dir']=clf_XX_3_gbm.predict(training_2.\
drop(['Policy_Number','Next_Premium','segment_predict'],axis=1))

In [91]:
training_2_ensemble['Next_Premium_Predict']=\
clf_linear.predict(training_2_ensemble[['Next_Premium_Predict_diff','Next_Premium_Predict_log','Next_Premium_Predict_dir']])

In [92]:
training_1=training_1[['Policy_Number','Next_Premium','Next_Premium_Predict']]
training_2_ensemble=training_2_ensemble[['Policy_Number','Next_Premium','Next_Premium_Predict']]
training_3=training_3[['Policy_Number','Next_Premium','Next_Premium_Predict']]
Ans_train=pd.concat([training_1,training_2_ensemble,training_3],axis=0)

In [93]:
Ans_train.shape

(210763, 3)

In [94]:
mean_absolute_error(Ans_train['Next_Premium'],Ans_train['Next_Premium_Predict'])

1648.1802103321443

產生上傳用test預測值

In [95]:
segment_predict_prob_test=clf_gbm.predict(testing_X)
segment_predict_prob_test=pd.DataFrame(segment_predict_prob_test,columns=['Prob0','Prob1'])

segment_predict_test=segment_predict_prob_test['Prob0'].apply(lambda x:0 if x>0.5 else 1)

testing_set_var_new=testing_set_var.copy().reset_index(drop=True)
testing_set_var_new['segment_predict']=segment_predict_test

In [96]:
testing_set_var_new['segment_predict'].value_counts()

1    129218
0      5779
Name: segment_predict, dtype: int64

In [97]:
testing_1=testing_set_var_new[testing_set_var_new['segment_predict']==0].reset_index(drop=True)
testing_2=testing_set_var_new[testing_set_var_new['segment_predict']==1].reset_index(drop=True)

In [98]:
testing_1['Next_Premium_Predict']=0

testing_3=testing_set_var_0.copy()
testing_3['Next_Premium_Predict']=0

In [99]:
#產生diff版的預測值
testing_2_ensemble=testing_2.copy()

testing_2_ensemble['Next_Premium_Predict_diff']=clf_XX_2_gbm.predict(testing_2.\
drop(['Policy_Number','Next_Premium','segment_predict'],axis=1))

#testing_2_ensemble=testing_2_ensemble.merge(baseline,on='Policy_Number',how='left')

testing_2_ensemble['Next_Premium_Predict_diff']=testing_2_ensemble['Premium_SUM']+testing_2_ensemble['Next_Premium_Predict_diff']
testing_2_ensemble['Next_Premium_Predict_diff']=testing_2_ensemble['Next_Premium_Predict_diff'].apply(lambda x: x if x>0 else 0)

In [100]:
#產生log版的預測值
testing_2_ensemble['Next_Premium_Predict_log']=clf_XX_gbm.predict(testing_2.\
drop(['Policy_Number','Next_Premium','segment_predict'],axis=1))

testing_2_ensemble['Next_Premium_Predict_log']=testing_2_ensemble['Next_Premium_Predict_log'].apply(lambda x: np.e**x-1)

In [101]:
#產生dir版的預測值
testing_2_ensemble['Next_Premium_Predict_dir']=clf_XX_3_gbm.predict(testing_2.\
drop(['Policy_Number','Next_Premium','segment_predict'],axis=1))

In [102]:
testing_2_ensemble['Next_Premium_Predict']=clf_linear.\
predict(testing_2_ensemble[['Next_Premium_Predict_diff','Next_Premium_Predict_log','Next_Premium_Predict_dir']])

In [103]:
testing_1=testing_1[['Policy_Number','Next_Premium_Predict']]
testing_2_ensemble=testing_2_ensemble[['Policy_Number','Next_Premium_Predict']]
testing_3=testing_3[['Policy_Number','Next_Premium_Predict']]
Ans=pd.concat([testing_1,testing_2_ensemble,testing_3],axis=0)

In [104]:
#產生上傳檔
Ans=Ans.rename(columns={'Next_Premium_Predict':'Next_Premium'})
Ans.to_csv('V2_9_5_4.csv',index=True)